Data Cleaning

In [ ]:
import pandas as pd

# -----------------------------
# Load CSV
# -----------------------------
df = pd.read_csv("users.csv")

# Remove duplicates
df = df.drop_duplicates()

# Clean column names
df.columns = df.columns.str.strip()

# Clean text columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()

df["is_active"] = df["is_active"].astype(str).str.lower().map({
    "true": 1,
    "false": 0
})

# Handle missing values
for col in df.columns:
    if df[col].dtype != "object":
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna("Unknown")

# Define date columns to process
watch_date = ["subscription_start_date", "created_at"]

for col in watch_date:
    if col in df.columns:
        df[col] = (
            pd.to_datetime(
                df[col],
                errors="coerce",     # invalid dates → NULL (SQL-safe)
                dayfirst=True        # important for Indian-style dates
            )
            .dt.strftime("%Y-%m-%d %H:%M:%S")  # SQL DATETIME format
        )


# 1. Standardize date formats (assuming DD-MM-YYYY to YYYY-MM-DD)
df['subscription_start_date'] = pd.to_datetime(
    df['subscription_start_date'],
    dayfirst=True,
    errors='coerce'
).dt.strftime('%Y-%m-%d')

# 2. Handle mixed data types: Convert numeric columns to proper numeric types
numeric_columns = ['age', 'monthly_spend', 'household_size']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. Fill or remove null values
# For this example, we'll fill nulls with a placeholder or drop rows if necessary
df['email'] = df['email'].fillna('unknown@example.com')
df = df.dropna(subset=['user_id'])  # Ensure 'user_id' is not null

# 4. Ensure consistent encoding: Convert to UTF-8 if necessary
df = df.applymap(lambda x: x.encode('utf-8').decode('utf-8') if isinstance(x, str) else x)

# Save the cleaned dataset
df.to_csv = 'users_cleaned_fixed.csv'

users.csv cleaned successfully!
